# Tratamento de dados

FONTE: [Dados abertos do CEAPS](https://www12.senado.leg.br/transparencia/dados-abertos-transparencia/dados-abertos-ceaps?utm_source=ActiveCampaign&utm_medium=email&utm_content=%237DaysOfCode+-+Ci%C3%AAncia+de+Dados+1%2F7%3A+Data+Cleaning+and+Preparation&utm_campaign=%5BAlura+%237Days+Of+Code%5D%28Java%29+Dia+1%2F7%3A+Consumir+uma+API+de+filmes). 
- Foram extraídos os dados dos anos de 2010 a 2022.
- Os dados foram baixados no dia 21/04/2022



Vamos começar extraindo os dados dos csv fazendo a leitura com pandas. Todas as planilhas baixadas serão unidas em um único dataset.

Após alguns testes iniciais, observou-se que deveriam ser usados as seguintes opções na função `read_csv`:
- sep=';' pois os dados estão separados por ponto e vírgula
- enconding='ISO-8859-1' pois os dados estão em português e pode haver caracteres especiais dessa língua, como acentuação
- skiprows=1 pois a primeira linha do arquivo é somente uma informação de quando foi a última vez que os dados foram atualizados
- decimal=',' separa números após a vírgula como sendo decimais
- thousands='.' o ponto separa as unidades de milhar

In [1]:
import pandas as pd
import os
from datetime import datetime

In [2]:
os.chdir(r"C:\Users\mathe\OneDrive\Documentos\Ciencia de Dados\projetos\7 days of code\dia 1 - limpeza de dados")
os.chdir('data')     # acessando diretório escolhido

data_list = list(filter(os.path.isfile, os.listdir()))      # faz uma lista com todos os arquivos da pasta

# lendo a primeira planilha
ds = pd.read_csv(data_list[0], sep=';', encoding='ISO-8859-1', skiprows=1, decimal=',', thousands='.')

# lendo as demais planilhas e concatenando com ds
for data in data_list[1:]:
    ds_concat = pd.read_csv(data, sep=';', encoding='ISO-8859-1', skiprows=1, decimal=',', thousands='.')
    ds = pd.concat([ds, ds_concat], axis=0, ignore_index=True)

display(ds.head())
ds.shape

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2010,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,CENTRAIS ELÉTRICAS DE RONDÔNIA,045216-3,19/01/2010,NaN,59.08,277219.0
1,2010,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,CENTRAIS ELÉTRICAS DE RONDônia,0454206-1,19/01/2010,NaN,40.47,277218.0
2,2010,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,NaN,12/01/2010,NaN,5000.0,229109.0
3,2010,1,ACIR GURGACZ,Divulgação da atividade parlamentar,09.040.457/0001-02,CENTRAL RONDÔNIA.COM,000003,05/01/2010,NaN,1500.0,268264.0
4,2010,1,ACIR GURGACZ,"Locomoção, hospedagem, alimentação, combustíve...",15.896.509/0001-31,POSTO FORTALEZA,018215,07/01/2010,NaN,30.0,268265.0


(286388, 11)

## Dados duplicados e nulos

Começamos verificando se há alguma linha duplicada:

In [3]:
sum(ds.duplicated())

0

A última coluna parece ser um tipo de ID para cada documento do CEAPS, um ID único para cada linha do dataset. Podemos verificar isso:

In [4]:
sum(ds['COD_DOCUMENTO'].duplicated())

1

In [5]:
ds[ds['COD_DOCUMENTO'].duplicated(keep=False)]

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
131284,2015,8,EDUARDO AMORIM,"Passagens aéreas, aquáticas e terrestres nacio...",13.353.495/0001-84,PROPAGTUR - Propag Turismo Ltda,"SEM FATURA"";02/08/2015""","Companhia Aérea: TAM, Localizador: YXGDSJ. Pas...","469,53",2014675.0,NaN
131291,2015,8,EDUARDO AMORIM,"Passagens aéreas, aquáticas e terrestres nacio...",13.353.495/0001-84,PROPAGTUR - Propag Turismo Ltda,"¨¨SEM FATURA"";17/08/2015""","Companhia Aérea: AVIANCA, Localizador: ZNEU9F....","460,45",2016557.0,NaN


In [6]:
ds[ds['COD_DOCUMENTO'].duplicated(keep=False)].index[0]

131284

Podemos perceber que a coluna COD_DOCUMENTO ficou sem dados nesses índices, mas isso é por conta de uma má formatação do arquivo. Nesses índices, a coluna DOCUMENTO ficou com 2 informações separadas por ";".

O jeito mais rápido de concertar isso é colocando os valores corretos nas colunas manualmente

In [7]:
id1 = ds[ds['COD_DOCUMENTO'].duplicated(keep=False)].index[0]
id2 = ds[ds['COD_DOCUMENTO'].duplicated(keep=False)].index[1]

ds.iloc[id1, 10] = ds.iloc[id1, 9]
ds.iloc[id1, 9] = ds.iloc[id1, 8]
ds.iloc[id1, 8] = ds.iloc[id1, 7]
ds.iloc[id1, 7] = "02/08/2015"
ds.iloc[id1, 6] = "SEM FATURA"

ds.iloc[id2, 10] = ds.iloc[id2, 9]
ds.iloc[id2, 9] = ds.iloc[id2, 8]
ds.iloc[id2, 8] = ds.iloc[id2, 7]
ds.iloc[id2, 7] = "17/08/2015"
ds.iloc[id2, 6] = "SEM FATURA"

ds[ds['COD_DOCUMENTO'].duplicated(keep=False)]

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO


Agora podemos ver que não há mais IDs de documento duplicados.
Verificando se há mais dados nulos no dataset:

In [8]:
ds.isnull().sum()/ds.shape[0]*100

ANO                   0.000000
MES                   0.000000
SENADOR               0.000000
TIPO_DESPESA          0.000000
CNPJ_CPF              0.000000
FORNECEDOR            0.000000
DOCUMENTO             6.809643
DATA                  0.000000
DETALHAMENTO         39.356747
VALOR_REEMBOLSADO     0.000000
COD_DOCUMENTO         0.000000
dtype: float64

Com o cálculo acima, podemos ver que há duas colunas com dados nulos:
- DOCUMENTO, que possui cerca de 6,8% dos seus dados vazios
- DETALHAMENTO, que possui quase 40% dos seus dados vazios.

A coluna DETALHAMENTO, quando analisada mais atentamente, nos mostra informações a mais sobre o tipo de despesa do senador. Isso pode ser útil em uma análise exploratória ou aplicação de modelo. O que será decidido aqui é preencher os dados vazios dessa coluna com os dados da coluna TIPO_DESPESA. Assim, sempre teremos alguma informação na coluna DETALHAMENTO, mesmo que ela não seja a mais específica possível.

Por hora, não se vê a necessidade de se saber qual é o documento que registra cada despesa dos senadores, por isso será decidido excluir a coluna DOCUMENTO. A coluna COD_DOCUMENTO também será excluída, pois não acrescenta em nada nas informações sobre os gastos.

In [9]:
ds_original = ds

# excluindo colunas
ds = ds.drop(columns = ['DOCUMENTO', 'COD_DOCUMENTO'])

# preenchendo dados nulos da coluna DETALHAMENTO
ds.loc[ds['DETALHAMENTO'].isnull(), 'DETALHAMENTO'] = ds[ds['DETALHAMENTO'].isnull()]['TIPO_DESPESA']

In [10]:
ds.isnull().sum().sum()

0

Como podemos ver, agora não resta mais nenhum dado nulo ou duplicado.

## Definindo os tipos de dados

Podemos executar o método `.info()` que nos dá algumas informações sobre os tipos de dados das colunas:

In [11]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286388 entries, 0 to 286387
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ANO                286388 non-null  int64 
 1   MES                286388 non-null  int64 
 2   SENADOR            286388 non-null  object
 3   TIPO_DESPESA       286388 non-null  object
 4   CNPJ_CPF           286388 non-null  object
 5   FORNECEDOR         286388 non-null  object
 6   DATA               286388 non-null  object
 7   DETALHAMENTO       286388 non-null  object
 8   VALOR_REEMBOLSADO  286388 non-null  object
dtypes: int64(2), object(7)
memory usage: 19.7+ MB


Logo percebemos 2 colunas que estão com o tipo dos seus dados de forma errada:
- DATA, que deveria estar em `Datetype`
- VALOR_REEMBOLSO, que deveria estar como `float`, por representar valor financeiro

In [12]:
ds['DATA'] = ds['DATA'].map(lambda x: datetime.strptime(x, '%d/%m/%Y'))

def format_num(num):
    if type(num) == 'str':
        num = num.replace(',', '.')
        return float(num)
    else:
        return num

ds['VALOR_REEMBOLSADO'] = ds['VALOR_REEMBOLSADO'].map(format_num)

Com as devidas alterações feitas, precisamos exportar esse dataset para um novo arquivo csv:

In [28]:
ds.to_csv('dados_tratados_CEAPS.csv', sep=';')